# From Notebook to ModelOp Center:
## Training, Evaluating, and Conforming a Model for Deployment


In this notebook, we demonstrate the process of 
1. training a model, 
2. evaluating its performace, 
3. saving it for later use,
4. and conforming it to MOC standards.

More specifically, we will train a logistic regression classifier on the German Credit Data dataset.

**I - Model Training**

Let's begin by loading relevant libraries. We will need `sklearn` for model training, and `aequitas` for bias detection.

In [1]:
import csv
import json
import pickle
import numpy as np
import pandas as pd
import shap

from aequitas.bias import Bias
from aequitas.group import Group
from aequitas.preprocessing import preprocess_input_df

from sklearn import set_config
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, \
                            f1_score, fbeta_score, confusion_matrix

# set_config(display='diagram')
pd.options.mode.chained_assignment = None

The **German Credit Data** dataset can be found here: https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data). Download it and load it from a *CSV* file. For our purposes, the dataset has been modified slightly to include an `id` column, and a `gender` column (engineered from `status_sex`, used to demonstarte bias). The target variable is under `label`. We have mapped the labels `[1,2]` to `[0,1]`, where `1` indicates the positive class (loan default).

In [2]:
data = pd.read_csv("german_credit_data.csv")

In [3]:
data.columns.values

array(['id', 'duration_months', 'credit_amount', 'installment_rate',
       'present_residence_since', 'age_years', 'number_existing_credits',
       'checking_status', 'credit_history', 'purpose', 'savings_account',
       'present_employment_since', 'debtors_guarantors', 'property',
       'installment_plans', 'housing', 'job', 'number_people_liable',
       'telephone', 'foreign_worker', 'gender', 'label'], dtype=object)

Let's look at some data:

In [4]:
data.head()

,id,duration_months,credit_amount,installment_rate,present_residence_since,age_years,number_existing_credits,checking_status,credit_history,purpose,...,debtors_guarantors,property,installment_plans,housing,job,number_people_liable,telephone,foreign_worker,gender,label
0,0,6,1169,4,4,67,2,A11,A34,A43,...,A101,A121,A143,A152,A173,1,A192,A201,male,0
1,1,48,5951,2,2,22,1,A12,A32,A43,...,A101,A121,A143,A152,A173,1,A191,A201,female,1
2,2,12,2096,2,3,49,1,A14,A34,A46,...,A101,A121,A143,A152,A172,2,A191,A201,male,0
3,3,42,7882,2,4,45,1,A11,A32,A42,...,A103,A122,A143,A153,A173,2,A191,A201,male,0
4,4,24,4870,3,4,53,2,A11,A33,A40,...,A101,A124,A143,A153,A173,2,A191,A201,male,1


Not all numeric columns need to be considered as numeric features. For example, `number_people_liable` only has two unique discrete values:

In [5]:
data.number_people_liable.value_counts()

1    845
2    155
Name: number_people_liable, dtype: int64

We may therefore treat it as a categorical feature. Note, however, that we may need to reconsider this option if more values appear in testing phases.

In [6]:
data.number_people_liable = data.number_people_liable.astype('object')

Before proceeding any further with model development, let us split the original dataset into two sets: a **baseline** set that will be used as a reference set, and a **sample** set which will mimic input data to the model once the model is in use.

In [7]:
df_baseline, df_sample = train_test_split(data, train_size=0.8, random_state=0)

df_baseline.to_json('df_baseline.json', orient='records', lines=True)
df_sample.to_json('df_sample.json', orient='records', lines=True)

Our data still contains non-predictive features, such as `id`, `label` and `gender` (excluded to remove explicit bias). We remove these below.

In [8]:
predictive_features = [
    f for f in list(data.columns.values) 
    if f not in ['id', 'label', 'gender']
]

As a sanity check, let us see which features are automatically encoded as **numeric**, and which are encoded as **categorical**.

In [9]:
categorical_features = [
    f for f in list(data.select_dtypes(include=['category', 'object'])) 
    if f in predictive_features
]

numeric_features = [
    f for f in predictive_features 
    if f not in categorical_features
]

**Categorical features**:

In [10]:
print(categorical_features)

['checking_status', 'credit_history', 'purpose', 'savings_account', 'present_employment_since', 'debtors_guarantors', 'property', 'installment_plans', 'housing', 'job', 'number_people_liable', 'telephone', 'foreign_worker']


**Numeric features**:

In [11]:
print(numeric_features)

['duration_months', 'credit_amount', 'installment_rate', 'present_residence_since', 'age_years', 'number_existing_credits']


Everything looks good; let us proceed with training. We need to specify **predictive** and **response** variables for each of the training and test sets. We set these by filtering the baseline and sample sets.

In [12]:
X_train = df_baseline[predictive_features]
X_test = df_sample[predictive_features]

y_train = df_baseline['label']
y_test = df_sample['label']

X_train.to_json('X_train.json', orient='records', lines=True)
X_test.to_json('X_test.json', orient='records', lines=True)

We will train a **Logistic Regression** classifier. Since our data contains categorical features, we will need to encode the data first. We'll use `pd.get_dummies()` for this purpose.

In [13]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# saving the final list of columns from getting dummies on X_train
predictive_features = list(X_train.columns)
pickle.dump(predictive_features, open('predictive_features.pickle', 'wb'))

We may now fit the classifier to the training data. Since "it is worse to classify a customer as good when they are bad, than it is to classify a customer as bad when they are good", we will use an **F_beta metric**, with `beta=2`, to judge the performance of our model.

In [14]:
# train logistic regression model
logreg = LogisticRegressionCV(
    Cs=50,
    cv=5,
    max_iter=1000,
    n_jobs=-1,
    scoring=make_scorer(fbeta_score, beta=2),
    class_weight='balanced'
)
logreg.fit(X_train, y_train)

LogisticRegressionCV(Cs=50, class_weight='balanced', cv=5, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=1000, multi_class='auto', n_jobs=-1, penalty='l2',
                     random_state=None, refit=True,
                     scoring=make_scorer(fbeta_score, beta=2), solver='lbfgs',
                     tol=0.0001, verbose=0)

In [15]:
logreg.score(X_train, y_train)

0.7373271889400922

In [16]:
logreg.score(X_test, y_test)

0.609375

**II - Model Evaluation**

Before saving our trained model for further use, let's look at some performance metrics. We will evaluate the model on both the training and test sets; we would like to see a stable performance.

For repeatability, let's define a function which computes multiple metrics at-a-time:

In [17]:
def compute_metrics(y, y_preds):
    """
    A function to evaluate a classification model
    
    param: y: true (actual) labels
    param: y_preds: predicted labels (as scored by model)
    
    return: mutiple classification performance metrics
    """
    
    return [
        accuracy_score(y, y_preds),
        precision_score(y, y_preds),
        recall_score(y, y_preds),
        f1_score(y, y_preds),
        fbeta_score(y, y_preds, beta=2),
    ]

Let us now compute predictions on both training and test sets:

In [18]:
y_test_preds = logreg.predict(X_test)
y_train_preds = logreg.predict(X_train)

We will display performance metrics in a DataFrame:

In [19]:
preformance_df = pd.DataFrame(
    data=[{}],
    columns=['Accuracy', 'Precision', 'Recall', 'F1 score', 'F2 Score'],
    index=['Training Set', 'Test Set']
)

In [20]:
preformance_df.loc['Training Set',:] = compute_metrics(y=y_train, y_preds=y_train_preds)
preformance_df.loc['Test Set',:] = compute_metrics(y=y_test, y_preds=y_test_preds)

Here's how our model performed:

In [21]:
preformance_df

,Accuracy,Precision,Recall,F1 score,F2 Score
Training Set,0.76,0.574850,0.793388,0.666667,0.737327
Test Set,0.66,0.443182,0.672414,0.534247,0.609375


While it's good to see that the performance on the training set is not too far off from the performance on the test set, further model improvements are needed to achieve better F2 scores. For now, we will contend with this model and use it to produce new predictions.

**III - Saving and Loading the Trained Model**

Now that the model is **trained** and **evaluated**, we save it in a binary format. It will then be loaded and used to make new predictions.

In [22]:
pickle.dump(logreg, open("logreg_classifier.pickle", 'wb'))

The model is reloaded on-demand as follows:

In [23]:
logreg_classifier = pickle.load(open("logreg_classifier.pickle", 'rb'))

Predictions are produced on-demand by calling the `predict()` function:

In [24]:
new_preds = logreg_classifier.predict(X_test)

**IV - Using a SHAP explainer model to explain feature importance**

Shap is used to explain feature importance, a functionality that is supported by ModelOp Center. We'll train an `explainer` and pickle it for use later.

In [25]:
# training the explainer on the model and train data
explainer = shap.LinearExplainer(logreg_classifier, X_train, feature_name=X_train.columns)

# getting shap values for the test data
shap_values = explainer.shap_values(X_test)

# re-organizing and sorting the data
shap_values = np.mean(abs(shap_values), axis=0).tolist()
shap_values = dict(zip(X_train.columns, shap_values))
sorted_shap_values = {k:v for k, v in sorted(shap_values.items(),
                                             key=lambda x: x[1])}

# show the values
sorted_shap_values

{'purpose_A410': 0.0011015123361155987,
 'purpose_A48': 0.0012967531186528689,
 'purpose_A44': 0.001462711487048438,
 'job_A171': 0.0017967294537423852,
 'debtors_guarantors_A101': 0.003388592277037583,
 'savings_account_A63': 0.004386178978108516,
 'purpose_A45': 0.005461134391834057,
 'job_A172': 0.005686354080995211,
 'number_people_liable_2': 0.008003072895393171,
 'property_A123': 0.010630772378958445,
 'savings_account_A64': 0.010659280922723148,
 'present_employment_since_A71': 0.010852584741487809,
 'purpose_A49': 0.011002198752194658,
 'property_A122': 0.011200962342052114,
 'installment_plans_A142': 0.011636582161781157,
 'foreign_worker_A201': 0.012050617217066623,
 'checking_status_A13': 0.013416249194786144,
 'savings_account_A62': 0.016635927414673724,
 'installment_plans_A141': 0.017022729668872392,
 'purpose_A42': 0.01779294007932323,
 'debtors_guarantors_A102': 0.01803372917561362,
 'present_residence_since': 0.01879768015436851,
 'foreign_worker_A202': 0.0218109004654

In [26]:
pickle.dump(explainer, open('explainer.pickle', 'wb'))

**V - Saving `_scored.json` datasets for further analysis**

The monitoring libraries need `score` (predictions), `label_values` (ground truth), and `predicted_probs` (probability outputs from predictions) features to work correctly. To that end, let us produce the necessary prediction outputs and append them to our labeled baseline and sample sets.

In [27]:
df_baseline_scored = df_baseline.copy(deep=True)
df_sample_scored = df_sample.copy(deep=True)

# recording scores (predictions)
df_baseline_scored["score"] = logreg_classifier.predict(
    pd.get_dummies(df_baseline)[predictive_features])
# recording predicted probabilities
df_baseline_scored['predicted_probs'] = [y for x, y in logreg_classifier.predict_proba(
    pd.get_dummies(df_baseline)[predictive_features])]

# recording scores (predictions)
df_sample_scored["score"] = logreg_classifier.predict(
    pd.get_dummies(df_sample)[predictive_features])
df_sample_scored['predicted_probs'] = [y for x, y in logreg_classifier.predict_proba(
    pd.get_dummies(df_sample)[predictive_features])]

The aequitas library requires the true label to be encoded as 'label_value', so let us rename that column.

In [28]:
df_baseline_scored.rename(columns={'label': 'label_value'}, inplace=True)
df_sample_scored.rename(columns={'label': 'label_value'}, inplace=True)

Let's save these two DataFrames before proceeding further:

In [29]:
df_baseline_scored.to_json('df_baseline_scored.json', orient='records', lines=True)
df_sample_scored.to_json('df_sample_scored.json', orient='records', lines=True)

**IV - Evaluating Bias on Protected Classes**

Since `gender` is a protected class, we have excluded from the list of predictive features. However, this does not guarantee that the model is not implicitely biased, as `gender` could potentially be inferred from other features. It is therefore imperative that we evaluate our model for Bias.

Now, we call the aequitas preprocessing function on our datasets, filtered to the features we care about: `score` (prediction), `label_value` (true label), and `gender` (protected class):

In [30]:
df_baseline_scored_processed, _ = preprocess_input_df(
    df_baseline_scored.loc[:,['score', 'label_value', 'gender']]
)
df_sample_scored_processed, _ = preprocess_input_df(
    df_sample_scored.loc[:,['score', 'label_value', 'gender']]
)

Let's start by computing some `Group` Metrics:

In [31]:
g_baseline, g_sample = Group(), Group()
xtab_baseline, _ = g_baseline.get_crosstabs(df_baseline_scored_processed)
xtab_sample, _ = g_sample.get_crosstabs(df_sample_scored_processed)

In [32]:
absolute_metrics_baseline = g_baseline.list_absolute_metrics(xtab_baseline)
absolute_metrics_sample = g_sample.list_absolute_metrics(xtab_sample)

Here are the absolute metrics, computed on baseline and sample sets, respectively:

In [33]:
xtab_baseline[['attribute_name', 'attribute_value'] + absolute_metrics_baseline].round(2)

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,gender,female,0.85,0.75,0.10,0.35,0.25,0.15,0.90,0.65,0.33,0.46,0.35
1,gender,male,0.77,0.75,0.11,0.46,0.25,0.23,0.89,0.54,0.67,0.40,0.28


In [34]:
xtab_sample[['attribute_name', 'attribute_value'] + absolute_metrics_sample].round(2)

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,gender,female,0.64,0.72,0.21,0.45,0.28,0.36,0.79,0.55,0.33,0.40,0.35
1,gender,male,0.70,0.62,0.14,0.61,0.38,0.30,0.86,0.39,0.67,0.46,0.26


We can also add some raw counts (group sizes) as follows:

In [35]:
xtab_baseline[[col for col in xtab_baseline.columns if col not in absolute_metrics_baseline]]

,model_id,score_threshold,k,attribute_name,attribute_value,pp,pn,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities
0,0,binary 0/1,334,gender,female,110,128,39,13,115,71,84,154,238,800
1,0,binary 0/1,334,gender,male,224,338,103,37,301,121,158,404,562,800


In [36]:
xtab_sample[[col for col in xtab_sample.columns if col not in absolute_metrics_sample]]

,model_id,score_threshold,k,attribute_name,attribute_value,pp,pn,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities
0,0,binary 0/1,88,gender,female,29,43,13,9,34,16,25,47,72,200
1,0,binary 0/1,88,gender,male,59,69,36,10,59,23,33,95,128,200


That's it for `Group` metrics. Let's move on to `Bias` metrics.

In [37]:
b_baseline, b_sample = Bias(), Bias()

bdf_baseline = b_baseline.get_disparity_predefined_groups(
    xtab_baseline, 
    original_df=df_baseline_scored_processed, 
    ref_groups_dict={'gender':'male'}, alpha=0.05, mask_significance=True
)

bdf_sample = b_sample.get_disparity_predefined_groups(
    xtab_sample, 
    original_df=df_sample_scored_processed, 
    ref_groups_dict={'gender':'male'}, alpha=0.05, mask_significance=True
)

get_disparity_predefined_group()
get_disparity_predefined_group()


We can now compute **disparity** metrics as follows

In [38]:
calculated_disparities_baseline = b_baseline.list_disparities(bdf_baseline)
calculated_disparities_sample = b_sample.list_disparities(bdf_sample)

disparity_metrics_df_baseline = bdf_baseline[
    ['attribute_name', 'attribute_value'] + \
        calculated_disparities_baseline
    ]
disparity_metrics_df_sample = bdf_sample[
    ['attribute_name', 'attribute_value'] + \
        calculated_disparities_sample
    ]

Here are the computed disparity metrics on baseline and sample sets, respectively:

In [39]:
disparity_metrics_df_baseline

,attribute_name,attribute_value,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity
0,gender,female,0.491071,1.159589,1.194891,0.77105,0.927787,0.993317,0.660875,1.103699,1.002287,1.008877
1,gender,male,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [40]:
disparity_metrics_df_sample

,attribute_name,attribute_value,ppr_disparity,pprev_disparity,precision_disparity,fdr_disparity,for_disparity,fpr_disparity,fnr_disparity,tpr_disparity,tnr_disparity,npv_disparity
0,gender,female,0.491525,0.873823,1.415292,0.734674,1.444186,0.729905,1.188,0.918261,1.164803,0.924714
1,gender,male,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000,1.000000,1.000000,1.000000


Some of the disparity metrics above are worrisome! We might need to retrain the model, possibly with better feature engineering. That's an exercise for a later time.

**V - Conforming Model Code to MOC Requirements**

Conformace is best-demonstrated through and example. Let's look at the code below:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import copy
import shap
from aequitas.preprocessing import preprocess_input_df
from aequitas.group import Group
from aequitas.bias import Bias
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, confusion_matrix
from scipy.spatial.distance import jensenshannon
from scipy.stats import epps_singleton_2samp, gaussian_kde, ks_2samp


# modelop.init
def begin():

    global logreg_classifier
    global predictive_features
    global df_baseline
    global explainer

    # load pickled logistic regression model
    logreg_classifier = pickle.load(open("logreg_classifier.pickle", "rb"))

    # load pickled predictive feature list
    predictive_features = pickle.load(open("predictive_features.pickle", "rb"))

    # load df_baseline_scored for metrics (specifically drift calculations)
    df_baseline = pd.read_json("df_baseline_scored.json", orient="records", lines=True)

    # load shap explainer
    explainer = pickle.load(open("explainer.pickle", "rb"))


def preprocess(data):
    # There are only two unique values in data.number_people_liable.
    # Treat it as a categorical feature
    data["number_people_liable"] = data["number_people_liable"].astype("object")

    # one-hot encode data with pd.get_dummies()
    data = pd.get_dummies(data)

    # in case features don't exist that are needed for the model (possible when dummying)
    # will create column of zeros for that feature
    for col in predictive_features:
        if col not in data.columns:
            data[col] = np.zeros(data.shape[0])

    return data


# modelop.score
def action(data):

    # Turn data into DataFrame
    data = pd.DataFrame(data)

    # preprocess data
    data = preprocess(data)

    # generate predictions
    data["predicted_score"] = logreg_classifier.predict(data[predictive_features])
    data["predicted_probs"] = [
        x[1] for x in logreg_classifier.predict_proba(data[predictive_features])
    ]

    # MOC expects the action function to be a *yield* function
    return data.to_dict(orient="records")
    # yield data.to_dict(orient="records")


# modelop.metrics
def metrics(data):
    # dictionary to hold final metrics
    metrics = {}

    # convert data into DataFrame
    data = pd.DataFrame(data)

    # getting dummies for shap values
    data_processed = preprocess(data)[predictive_features]

    # calculate metrics
    f1 = f1_score(data["label_value"], data["score"])
    cm = confusion_matrix(data["label_value"], data["score"])
    labels = ["Default", "Pay Off"]
    cm = matrix_to_dicts(cm, labels)
    fpr, tpr, thres = roc_curve(data["label_value"], data["predicted_probs"])
    auc_val = roc_auc_score(data["label_value"], data["predicted_probs"])
    rc = [{"fpr": x[0], "tpr": x[1]} for x in list(zip(fpr, tpr))]

    # categorical/numerical columns for drift
    categorical_features = [
        f
        for f in list(metrics_sample.select_dtypes(include=["category", "object"]))
        if f in df_baseline.columns
    ]
    numerical_features = [
        f for f in df_baseline.columns if f not in categorical_features
    ]
    numerical_features = [
        x
        for x in numerical_features
        if x not in ["id", "score", "label_value", "predicted_probs"]
    ]

    # assigning metrics to output dictionary
    metrics["f1_score"] = f1
    metrics["confusion_matrix"] = cm
    metrics["auc"] = auc_val
    metrics["ROC"] = rc
    metrics["bias"] = get_bias_metrics(data)
    metrics["drift_metrics"] = get_drift_metrics(
        df_baseline, data, numerical_features, categorical_features
    )
    metrics["shap"] = get_shap_values(data_processed)

    # MOC expects the action function to be a *yield* function
    # yield metrics
    return metrics


def get_bias_metrics(data):
    # To measure Bias towards gender, filter DataFrame
    # to "score", "label_value" (ground truth), and
    # "gender" (protected attribute)
    data_scored = data[["score", "label_value", "gender"]]

    # Process DataFrame
    data_scored_processed, _ = preprocess_input_df(data_scored)

    # Group Metrics
    g = Group()
    xtab, _ = g.get_crosstabs(data_scored_processed)

    # Absolute metrics, such as 'tpr', 'tnr','precision', etc.
    absolute_metrics = g.list_absolute_metrics(xtab)

    # DataFrame of calculated absolute metrics for each sample population group
    absolute_metrics_df = xtab[
        ["attribute_name", "attribute_value"] + absolute_metrics
    ].round(2)

    # Bias Metrics
    b = Bias()

    # Disparities calculated in relation gender for "male" and "female"
    bias_df = b.get_disparity_predefined_groups(
        xtab,
        original_df=data_scored_processed,
        ref_groups_dict={"gender": "male"},
        alpha=0.05,
        mask_significance=True,
    )

    # Disparity metrics added to bias DataFrame
    calculated_disparities = b.list_disparities(bias_df)

    disparity_metrics_df = bias_df[
        ["attribute_name", "attribute_value"] + calculated_disparities
    ]

    output_metrics_df = disparity_metrics_df  # or absolute_metrics_df

    # Output a JSON object of calculated metrics
    return output_metrics_df.to_dict(orient="records")


def matrix_to_dicts(matrix, labels):
    cm = []
    for idx, label in enumerate(labels):
        cm.append(dict(zip(labels, matrix[idx, :].tolist())))
    return cm


def get_drift_metrics(df_baseline, df_sample, numerical_cols, categorical_cols):
    drift_metrics = {}
    drift_metrics["drift__es"] = es_metric(df_baseline, df_sample, numerical_cols)
    drift_metrics["drift__ks"] = ks_metric(df_baseline, df_sample, numerical_cols)
    drift_metrics["drift__js"] = js_metric(
        df_baseline, df_sample, numerical_cols, categorical_cols
    )
    drift_metrics["concept_drift__es"] = es_metric(df_baseline, df_sample, ["score"])
    drift_metrics["concept_drift__ks"] = ks_metric(df_baseline, df_sample, ["score"])
    drift_metrics["concept_drift__js"] = js_metric(
        df_baseline, df_sample, ["score"], []
    )
    return drift_metrics


def ks_metric(df1, df2, numerical_columns):
    ks_tests = [
        ks_2samp(data1=df1.loc[:, col], data2=df2.loc[:, col])
        for col in numerical_columns
    ]
    p_values = [x[1] for x in ks_tests]
    list_of_pval = [f"{col}_p-value" for col in numerical_columns]
    ks_pvalues = dict(zip(list_of_pval, p_values))
    return ks_pvalues


def es_metric(df1, df2, numerical_columns):
    es_tests = []
    for col in numerical_columns:
        try:
            es_test = epps_singleton_2samp(x=df1.loc[:, col], y=df2.loc[:, col])
        except np.linalg.LinAlgError:
            es_test = [None, None]
        es_tests.append(es_test)
    p_values = [x[1] for x in es_tests]
    list_of_pval = [f"{col}_p-value" for col in numerical_columns]
    es_pvalues = dict(zip(list_of_pval, p_values))
    return es_pvalues


def js_metric(df1, df2, numerical_columns, categorical_columns):
    res = {}
    STEPS = 100
    for col in categorical_columns:
        col_baseline = df1[col].to_frame()
        col_sample = df2[col].to_frame()
        col_baseline["source"] = "baseline"
        col_sample["source"] = "sample"

        col_ = pd.concat([col_baseline, col_sample], ignore_index=True)

        arr = (
            col_.groupby([col, "source"])
            .size()
            .to_frame()
            .reset_index()
            .pivot(index=col, columns="source")
            .droplevel(0, axis=1)
        )
        arr_ = arr.div(arr.sum(axis=0), axis=1)
        arr_.fillna(0, inplace=True)
        js_distance = jensenshannon(
            arr_["baseline"].to_numpy(), arr_["sample"].to_numpy()
        )
        res.update({col: js_distance})

    for col in numerical_columns:
        # fit guassian_kde
        col_baseline = df1[col]
        col_sample = df2[col]
        kde_baseline = gaussian_kde(col_baseline)
        kde_sample = gaussian_kde(col_sample)

        # get range of values
        min_ = min(col_baseline.min(), col_sample.min())
        max_ = max(col_baseline.max(), col_sample.max())
        range_ = np.linspace(start=min_, stop=max_, num=STEPS)

        # sample range from KDE
        arr_baseline_ = kde_baseline(range_)
        arr_sample_ = kde_sample(range_)

        arr_baseline = arr_baseline_ / np.sum(arr_baseline_)
        arr_sample = arr_sample_ / np.sum(arr_sample_)

        # calculate js distance
        js_distance = jensenshannon(arr_baseline, arr_sample)

        res.update({col: js_distance})

    list_output = sorted(res.items(), key=lambda x: x[1], reverse=True)
    dict_output = dict(list_output)
    return dict_output


def get_shap_values(data):
    # getting shap values for the test data
    shap_values = explainer.shap_values(data)

    # re-organizing and sorting the data
    shap_values = np.mean(abs(shap_values), axis=0).tolist()
    shap_values = dict(zip(data.columns, shap_values))
    sorted_shap_values = {
        k: v for k, v in sorted(shap_values.items(), key=lambda x: x[1])
    }

    # show the values
    return sorted_shap_values

There are four main sections that are standard to almost any model in MOC:
1. Library imports
2. `init` function
3. `score` function
4. `metrics` function

**Library** imports are always at the top. We don't need to include all libraries that we used for training and model evaluation. We just need the libraries for processing and scoring.

The **`init`** function runs once per deployment, and is used to load and persist into memory any variable that needs to be accessed at scoring time. For example, the init function is where we load the saved model binary. We make the variable global so it can be accessed from the scoring function.

The **`score`** function is the function that runs anytime we make a scoring (prediction) request. This is where we put our prediction code. We have to remember to include any steps that were not captured by the pipeline, such as feature engineering or re-encoding.

The **`metrics`** functions is where model evaluation is carried out. In our example, this is the place where we replicate the calculations of Group and/or Bias metrics.

Let us test our source code to see if we missed anything. We will load input data and scored input data to test both the scoring and metrics functions:

In [2]:
test_sample = pd.read_json('df_baseline.json', lines=True, orient='records')
metrics_sample = pd.read_json('df_sample_scored.json', lines=True, orient='records')

Let's check that the **`init`** function can load the trained model binary:

In [3]:
begin()

No errors from the **`init`** function. Let us now call the **`score`** function on input data:

In [4]:
scores = action(test_sample)

In [5]:
pd.DataFrame(scores).head()

,id,duration_months,credit_amount,installment_rate,present_residence_since,age_years,number_existing_credits,label,checking_status_A11,checking_status_A12,...,number_people_liable_1,number_people_liable_2,telephone_A191,telephone_A192,foreign_worker_A201,foreign_worker_A202,gender_female,gender_male,predicted_score,predicted_probs
0,687,36,2862,4,3,30,1,0,0,1,...,1,0,1,0,1,0,0,1,1,0.804774
1,500,24,3123,4,1,27,1,1,1,0,...,1,0,1,0,1,0,1,0,1,0.878432
2,332,60,7408,4,2,24,1,1,0,1,...,1,0,1,0,1,0,1,0,1,0.958228
3,979,15,1264,2,2,25,1,1,0,1,...,1,0,1,0,1,0,0,1,1,0.754847
4,817,6,1554,1,2,24,2,0,0,0,...,1,0,0,1,1,0,1,0,0,0.080424


We have scores! Last but not least, let's call the **`metrics`** function on scored data:

In [6]:
metrics_output = metrics(metrics_sample)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
Estimated covariance matrix does not have full rank. This indicates a bad choice of the input t and the test might not be consistent.


get_disparity_predefined_group()


In [7]:
metrics_output

{'f1_score': 0.5342465753424657,
 'confusion_matrix': [{'Default': 93, 'Pay Off': 49},
  {'Default': 19, 'Pay Off': 39}],
 'auc': 0.7325157843613406,
 'ROC': [{'fpr': 0.0, 'tpr': 0.0},
  {'fpr': 0.0, 'tpr': 0.017241379310344827},
  {'fpr': 0.007042253521126761, 'tpr': 0.017241379310344827},
  {'fpr': 0.007042253521126761, 'tpr': 0.034482758620689655},
  {'fpr': 0.014084507042253521, 'tpr': 0.034482758620689655},
  {'fpr': 0.014084507042253521, 'tpr': 0.1206896551724138},
  {'fpr': 0.028169014084507043, 'tpr': 0.1206896551724138},
  {'fpr': 0.028169014084507043, 'tpr': 0.15517241379310345},
  {'fpr': 0.035211267605633804, 'tpr': 0.15517241379310345},
  {'fpr': 0.035211267605633804, 'tpr': 0.2413793103448276},
  {'fpr': 0.056338028169014086, 'tpr': 0.2413793103448276},
  {'fpr': 0.056338028169014086, 'tpr': 0.29310344827586204},
  {'fpr': 0.06338028169014084, 'tpr': 0.29310344827586204},
  {'fpr': 0.06338028169014084, 'tpr': 0.3103448275862069},
  {'fpr': 0.07042253521126761, 'tpr': 0.31

**Done and done!**